In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:17<00:00, 9.80MB/s]


In [2]:
import torch.nn as nn

class VGG16Net(nn.Module):
  def __init__(self, num_classes=10):
    super(VGG16Net, self).__init__()

    self.features = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1), #
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(64, 128, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(128, 256, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(256, 512, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(512, 512, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Linear(512 * 1 * 1, 4096), # Corrected input size
        nn.ReLU(),
        nn.Dropout(),

        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(),

        nn.Linear(4096, num_classes)
    )

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16Net(num_classes=10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
  model.train()
  for xb, yb in train_dataloader:
    xb, yb = xb.to(device), yb.to(device)
    pred = model(xb)
    loss = loss_fn(pred, yb)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for xb, yb in test_dataloader:
      xb, yb = xb.to(device), yb.to(device)
      pred = model(xb)
      _, predicted = torch.max(pred, 1)
      correct += (predicted==yb).sum().item()
      total += yb.size(0)

  accuracy = correct/total
  print(f"Epoch {epoch+1}: Test Accuracy = {accuracy:.4f}")